In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
os.makedirs('/root/.kaggle',exist_ok = True)
!cp /content/gdrive/My\ Drive/DeepLearning/kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c the-nature-conservancy-fisheries-monitoring

In [0]:
# 解压训练集数据
! unzip /content/train.zip

In [0]:
# 解压第一部分测试集数据
!unzip /content/test_stg1.zip

In [6]:
# 解压第二部分测试集数据
!7za x test_stg2.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1152137421 bytes (1099 MiB)

Extracting archive: test_stg2.7z
--
Path = test_stg2.7z
Type = 7z
Physical Size = 1152137421
Headers Size = 82081
Method = LZMA:25
Solid = +
Blocks = 1

  0%      0% - test_stg2/image_00001.jpg                                  0% 38 - test_stg2/image_00039.jpg                                     0% 80 - test_stg2/image_00081.jpg                                     1% 125 - test_stg2/image_00126.jpg                                    

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os,cv2
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
# 定义取图片的路径
TRAIN_DIR = '/content/train/'
TEST_DIR = '/content/test_stg2/'
WEIGHT_DIR = '/content/vgg16_weights.npz'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [0]:
# 获取图片的路径方法
def get_images(data_dir,fish):
    """Load files from train folder"""
    fish_dir = data_dir+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images
# 将图片读入内存
def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (224, 224), interpolation=cv2.INTER_CUBIC)
    return im

In [10]:
# 获取训练数据
files = []
y_all = []

for fish in FISH_CLASSES:
    fish_files = get_images(TRAIN_DIR,fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

1719 photos of ALB
200 photos of BET
117 photos of DOL
67 photos of LAG
465 photos of NoF
299 photos of OTHER
176 photos of SHARK
734 photos of YFT


In [11]:
# 将所有图片加载到内存中
X = np.ndarray((len(files), 224, 224, 3))

for i, im in enumerate(files): 
    X[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))
print(X.shape)

Processed 0 of 3777
Processed 1000 of 3777
Processed 2000 of 3777
Processed 3000 of 3777
(3777, 224, 224, 3)


In [0]:
# 转化为one-hot编码
y = LabelEncoder().fit_transform(y_all)
y = np_utils.to_categorical(y)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.2, random_state=23, 
                                                    stratify=y)

In [14]:
print(X_train.shape,X_valid.shape,y_train.shape,y_valid.shape)

(3021, 224, 224, 3) (756, 224, 224, 3) (3021, 8) (756, 8)


In [0]:
import numpy as np
from keras.layers import Input, Add, Dense,Activation,BatchNormalization, Flatten,Conv2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras import layers
from keras.layers import MaxPool2D

In [0]:
def identity_block(X, f, filters, stage, block):
    conv_name_base = 'res'+str(stage)+block+'_branch'
    bn_name_base = 'bn'+str(stage)+block+'_branch'
    
    F1, F2, F3 = filters
    X_shortcut = X
    
    # 第一层
    X = Conv2D(filters = F1,kernel_size = (1,1),strides = (1,1),padding='same',name=conv_name_base+'2a',
              kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base+'2a')(X)
    X = Activation('relu')(X)
    
    # 第二层
    X = Conv2D(filters = F2, kernel_size = (f ,f),strides = (1,1),padding='same',name=conv_name_base+'2b',
              kernel_initializer =glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base+'2b')(X)
    X = Activation('relu')(X)
    
    # 第三层
    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), padding='same',name=conv_name_base+'2c',
              kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name=bn_name_base+'2c')(X)
    
    X = layers.add([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [0]:
def convolutional_block(X, f, filters, stage, block, s=2):
    conv_name_base = 'res'+str(stage)+block+'_branch'
    bn_name_base = 'bn'+str(stage)+block+'_branch'
    
    F1, F2, F3 = filters
    X_shortcut = X
    
    # 第一层
    X = Conv2D(F1,(1,1),strides=(s,s),name=conv_name_base+'2a',padding='valid',
              kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)
    
    # 第二层
    X = Conv2D(F2,(f,f),strides=(1,1),name=conv_name_base+'2b',padding='same',
              kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)
    
    # 第三层
    X = Conv2D(F3,(1,1),strides=(1,1),name=conv_name_base+'2c',padding='valid',
              kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3,name=bn_name_base+'2c')(X)
    
    # short_path
    X_shortcut = Conv2D(F3,(1,1),strides=(s,s),name=conv_name_base+'1',padding='valid',
                       kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name=bn_name_base+'1')(X_shortcut)
    
    X = layers.add([X_shortcut,X])
    X =Activation('relu')(X)
    return X

In [0]:
def ResNet50(input_shape = (224, 224, 3), classes=8):
    X_input = Input(input_shape)
    
    # 第一层
    X = Conv2D(64,(7,7),strides = (2,2),name = 'conv1',padding='same',
               kernel_initializer=glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(axis = 3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPool2D((3,3),strides=(2,2),padding='same')(X)
    
    # 第二层
    X = convolutional_block(X, f=3, filters=[64,64,256],stage=2,block='a', s=1)
    X = identity_block(X, 3, [64,64,256],stage=2,block='b')
    X = identity_block(X, 3, [64,64,256],stage=2,block='c')
    
    # 第三层
    X = convolutional_block(X, f=3, filters=[128,128,512],stage=3,block='a', s=2)
    X = identity_block(X, 3, [128,128,512],stage=3,block='b')
    X = identity_block(X, 3, [128,128,512],stage=3,block='c')
    X = identity_block(X, 3, [128,128,512],stage=3,block='d')
    
    # 第四层
    X = convolutional_block(X, f=3, filters=[256,256,1024],stage=4,block='a', s=2)
    X = identity_block(X, 3, [256,256,1024],stage=4,block='b')
    X = identity_block(X, 3, [256,256,1024],stage=4,block='c')
    X = identity_block(X, 3, [256,256,1024],stage=4,block='d')
    X = identity_block(X, 3, [256,256,1024],stage=4,block='e')
    X = identity_block(X, 3, [256,256,1024],stage=4,block='f')
    
    # 第五层
    X = convolutional_block(X, f=3, filters=[512,512,2048],stage=5,block='a', s=2)
    X = identity_block(X, 3, [512,512,2048],stage=5,block='b')
    X = identity_block(X, 3, [512,512,2048],stage=5,block='c')
    
    X = layers.AveragePooling2D(pool_size=(7, 7), name="avg_pool")(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes),
    kernel_initializer = glorot_uniform(seed=0))(X)

    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    
    return model

In [0]:
model = ResNet50(input_shape=(224,224,3),classes=8)

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [44]:
model.fit(X_train,y_train,epochs = 10, batch_size = 32)

Epoch 1/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.0878 - acc: 0.9722
Epoch 2/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.1157 - acc: 0.9666
Epoch 3/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.0516 - acc: 0.9834
Epoch 4/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.1042 - acc: 0.9686
Epoch 5/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.0400 - acc: 0.9891
Epoch 6/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.0185 - acc: 0.9940
Epoch 7/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.0157 - acc: 0.9960
Epoch 8/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.0333 - acc: 0.9904
Epoch 9/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.0697 - acc: 0.9811
Epoch 10/10
3021/3021 [==============================] - 39s 13ms/step - loss: 0.1307 - acc: 0.9590


In [45]:
preds = model.evaluate(X_valid,y_valid)
print("Loss="+str(preds[0]))
print("Test Accuracy="+str(preds[1]))

756/756 [==============================] - 3s 5ms/step
Loss=5.223241740433627
Test Accuracy=0.5383597886751569


In [0]:
 test_file_path = [TEST_DIR+im for im in os.listdir(TEST_DIR)]

In [0]:
test_preds = np.zeros((12153,8))
test = np.ndarray((100, 224, 224, 3))

In [0]:
for i in range(121):
  print(i)
  for j in range(100):
    test[j] = read_image(test_file_path[i*100+j])
  t_pred = model.predict(test)
  test_preds[i*100:(i+1)*100,:] = t_pred
for i in range(53):
  test[j] = read_image(test_file_path[121*100+i])
t_pred = model.predict(test)
test_preds[121*100:121*100+53,:] = t_pred[0:53]

In [0]:
test_file_path1 = ['test_stg2/'+im for im in os.listdir(TEST_DIR)]
submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_file_path1)
submission.to_csv('submisision2.csv',index=None)

submission.head()

In [0]:
test_files = [im for im in os.listdir('/content/test_stg1')]
test = np.ndarray((len(test_files), 224, 224, 3))
for i, im in enumerate(test_files): 
    test[i] = read_image('/content/test_stg1/'+im)

In [0]:
test_preds = np.zeros((1000,8))
for i in range(10):
  print(i)
  t_pred = model.predict(test[i*100:(i+1)*100])
  test_preds[i*100:(i+1)*100,:] = t_pred



submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()


submission.to_csv('submisision1.csv',index=None)